# Исследование надёжности заёмщиков

**Цель исследования** - определить, как влияет семейное положение и наличие детей у клиента на своевременное погашение кредита. 

**Заказчик исследования** - отдел кредитования банка. 

Исследование базируется на статистике, предоставленной банком и отражающей платежеспособность клиентов. Результаты исследования будут применены при создании модели кредитного скоринга - системы, оценивающей способность заемщика вернуть кредит.

## Обзор данных

In [1]:
# Импортируем библиотеку pandas
import pandas as pd

In [2]:
# Прочитаем csv-файл
data = pd.read_csv('data_bank.csv')

In [3]:
# Выведим первые 10 строчек датафрейма data на экран.
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# Выведим основную информацию о датафрейме
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [5]:
# Выводим количество пропущенных значений для каждого столбца
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце мы обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. 

In [6]:
# Заполняем пропуски в этом столбце total_income медианным значением по каждому типу занятости из столбца income_type
for d in data['income_type'].unique():
    data.loc[(data['income_type'] == d) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[data['income_type'] == d, 'total_income'].median()

In [7]:
# Проверим, что пропусков в столбце total_income теперь нет
data['total_income'].isna().sum() 

0

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Далее обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.

In [8]:
# Обработаем значения в столбце days_employed: заменим все отрицательные значения положительными
data['days_employed'] = data['days_employed'].abs()

In [9]:
# Для каждого типа занятости выведим медианное значение трудового стажа days_employed в днях
data.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставляем их как есть.

In [10]:
# Выведим перечень уникальных значений столбца children
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [11]:
# В столбце children удаляем строки, в которых встречаются аномальные значения
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [12]:
# Выведим перечень уникальных значений столбца children, чтобы убедиться, что артефакты удалены
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

In [13]:
# Заполняем пропуски в столбце days_employed медианными значениями по каждому типу занятости income_type.
for d in data['income_type'].unique():
    # 1й вариант решения
    filter_income_type = (data['income_type'] == d)
    filter_empty_days_employed = data['days_employed'].isna()
    days_employed_median = data.loc[filter_income_type, 'days_employed'].median()
    
    data.loc[filter_income_type & filter_empty_days_employed, 'days_employed'] = days_employed_median
    
    # 2й вариант решения
    # data.loc[(data['income_type'] == d) & (data['days_employed'].isna()), 'days_employed'] = \
    #   data.loc[data['income_type'] == d, 'days_employed'].median()

In [14]:
# Проверим все колонки на наличие пропусков - убедимся, что в их отсутствии
data.isna().sum()  

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [15]:
# Выведем общую информацию о датасете и обратим внимание на тип данных в столбце total_income
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     21402 non-null  float64
 2   dob_years         21402 non-null  int64  
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      21402 non-null  float64
 11  purpose           21402 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


In [16]:
# Заменяем вещественный тип данных в столбце total_income на целочисленный с помощью метода astype()
data['total_income'] = data['total_income'].astype('int')

# Проверим, что тип данных поменялся
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     21402 non-null  float64
 2   dob_years         21402 non-null  int64  
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      21402 non-null  int32  
 11  purpose           21402 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 2.0+ MB


### Обработка дубликатов

In [17]:
# Обработаем неявные дубликаты в столбце education. В этом столбце есть одни и те же значения, но записанные по-разному: 
# с использованием заглавных и строчных букв. Приводим их нижнему регистру.
data['education'] = data['education'].str.lower()

In [18]:
# Выводим на экран количество строк-дубликатов в данных.
data.duplicated().sum()

71

In [19]:
# Удаляем дубликаты
data = data.drop_duplicates()
print(data.duplicated().sum())

0


In [20]:
# Создаем в датафрейме data столбец total_income_category с категориями:
# 0–30000 — 'E';
# 30001–50000 — 'D';
# 50001–200000 — 'C';
# 200001–1000000 — 'B';
# 1000001 и выше — 'A'.
# Создаем функцию categorize_income()
def categorize_income(total_income):
     if total_income <= 30000:
        return 'E'
     if total_income <= 50000:
        return 'D'
     if total_income <=200000:
        return 'C'
     if total_income <= 1000000:
        return 'B'
     return 'A'

In [21]:
# Применим функцию методом apply()
# Создаем новый столбец с категориями
data['total_income_category'] = data['total_income'].apply(categorize_income)
print(data.head(10))

   children  days_employed  dob_years education  education_id  \
0         1    8437.673028         42    высшее             0   
1         1    4024.803754         36   среднее             1   
2         0    5623.422610         33   среднее             1   
3         3    4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0     926.185831         27    высшее             0   
6         0    2879.202052         43    высшее             0   
7         0     152.779569         50   среднее             1   
8         2    6929.865299         35    высшее             0   
9         0    2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M

In [22]:
# Выведим на экран перечень уникальных целей взятия кредита из столбца purpose.
print(data['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


In [23]:
# Создаем функцию categorize_purpose(), которая на основании данных из столбца purpose сформирует новый столбец purpose_category, 
# в который войдут следующие категории:
# 1.'операции с автомобилем',
# 2.'операции с недвижимостью',
# 3.'проведение свадьбы',
# 4.'получение образования'.
def categorize_purpose(purpose):
    if purpose in ['приобретение автомобиля', 'на покупку подержанного автомобиля', \
                   'на покупку своего автомобиля', 'автомобили', 'сделка с подержанным автомобилем',\
                   'автомобиль', 'свой автомобиль', 'сделка с автомобилем', 'на покупку автомобиля']:
        return 'операции с автомобилем'
    if purpose in ['покупка жилья', 'операции с жильем', 'покупка жилья для семьи', \
                   'покупка недвижимости', 'покупка коммерческой недвижимости',\
                   'покупка жилой недвижимости', 'строительство собственной недвижимости',\
                   'недвижимость', 'строительство недвижимости', 'операции с коммерческой недвижимостью',\
                   'строительство жилой недвижимости', 'жилье', 'операции со своей недвижимостью',\
                   'покупка своего жилья', 'операции с недвижимостью', 'покупка жилья для сдачи', 'ремонт жилью']:
        return 'операции с недвижимостью'
    if purpose in ['сыграть свадьбу', 'на проведение свадьбы', 'свадьба']:
        return 'проведение свадьбы'
    if purpose in ['дополнительное образование', 'образование', 'заняться образованием', 'получение образования',\
                  'получение дополнительного образования', 'получение высшего образования',\
                   'профильное образование', 'высшее образование', 'заняться высшим образованием']:
        return 'получение образования'    

In [24]:
# Применим функцию методом apply()
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
print(data.head(10))

   children  days_employed  dob_years education  education_id  \
0         1    8437.673028         42    высшее             0   
1         1    4024.803754         36   среднее             1   
2         0    5623.422610         33   среднее             1   
3         3    4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0     926.185831         27    высшее             0   
6         0    2879.202052         43    высшее             0   
7         0     152.779569         50   среднее             1   
8         2    6929.865299         35    высшее             0   
9         0    2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M

In [25]:
# Сгруппируем датасет по количеству детей и рассчитаем для каждой группы среднее значение признака 
# наличия задолженности по возврату кредитов. Также посчитаем количество строк для каждой группы клиентов.
data.groupby('children')['debt'].agg(['mean', 'count'])

,mean,count
children,,
0,0.075438,14091
1,0.092346,4808
2,0.094542,2052
3,0.081818,330
4,0.097561,41
5,0.000000,9


Вывод: Прослеживается следующая зависимость между количеством детей и возвратом кредита в срок: заемщики без детей чаще возвращают кредиты в срок. При этом, в анализе мы не учитываем категории с тремя, четырьмя и пятью детьми, так как выборки нерепрезентативны, слишком мало значений, в сумме менее 2%.

In [26]:
# Сгруппируем датасет по семейному положению и рассчитаем для каждой группы среднее значение признака 
# наличия задолженности по возврату кредитов. Также посчитаем количество строк для каждой группы клиентов.
data.groupby('family_status')['debt'].agg(['mean', 'count'])

,mean,count
family_status,,
Не женат / не замужем,0.097639,2796
в разводе,0.070648,1189
вдовец / вдова,0.066246,951
гражданский брак,0.093130,4134
женат / замужем,0.075606,12261


Вывод: Прослеживается небольшая зависимость между семейным положением и возвратом кредита в срок:

- клиенты со статусами "Не женат / не замужем" и "гражданский брак" имеют самый высокий процент по невозврату кредитов.
- клиенты со статусом "вдовец / вдова" имеют самый низкий процент по невозврату кредитов.

In [27]:
# Сгруппируем датасет по категории дохода и рассчитаем для каждой группы среднее значение признака 
# наличия задолженности по возврату кредитов. Также посчитаем количество строк для каждой категории.
data.groupby('total_income_category')['debt'].agg(['mean', 'count'])

,mean,count
total_income_category,,
A,0.080000,25
B,0.070602,5014
C,0.084982,15921
D,0.060172,349
E,0.090909,22


Вывод: Категории с самым высоким уровнем дохода и самым низким можно не учитывать, так как выборка мала. В оставшихся категориях четкая зависимость не прослеживается.

In [28]:
# Сгруппируем датасет по категории цель кредита и рассчитаем для каждой группы среднее значение признака 
# наличия задолженности по возврату кредитов. Также посчитаем количество строк для каждой категории.
data.groupby('purpose_category')['debt'].agg(['mean', 'count'])

,mean,count
purpose_category,,
операции с автомобилем,0.093480,4279
операции с недвижимостью,0.072551,10751
получение образования,0.092528,3988
проведение свадьбы,0.079118,2313


Вывод: Прослеживается зависимость:

- самые ответственные заемщики - это клиенты из категорий "операции с недвижимостью" и "проведение свадьбы".
- хуже обстоят дела с категориями "операции с автомобилем" и "получение образования". Они чаще не возвращают кредиты.

Пропуски в данных могут возникнуть по следующим причинам:

1. Поля, в которых возникли пропуски, были необязательными к заполнению.
2. Некорректно были введены данные.
3. Ошибка оператора при переносе данных с бумажного носителя на электорнный.
4. Возможно, у респондента отсутствовала необходимая информация на момент заполнения анкеты/формы.
5. Ошибка в коде формы, из-за чего некорректно обрабатывается вводимая респондентом информация.

Заполнить пропуски медианным значением — лучшее решение для количественных переменных, так как оно учитывает большие всплески в значениях выборки, в отличие от среднего значения. При наличии больших всплесков в значениях, среднее арифмитическое сместит распределение.

В результате исследования датасета "data" были изучены данные, заполнены пропуски, удалены дубликаты. По результатам объединения клиентов по разным параметрам можно сделать выводы:

Прослеживается следующая зависимость между количеством детей и возвратом кредита в срок: заемщики без детей чаще возвращают кредиты в срок. При этом, в анализе мы не учитываем категории с тремя, четырьмя и пятью детьми, так как выборки нерепрезентативны, слишком мало значений, в сумме менее 2%.
Прослеживается нечеткая зависимость между семейным положением и возвратом кредита в срок:
клиенты со статусами "Не женат / не замужем" и "гражданский брак" имеют самый высокий процент по невозврату кредитов.
клиенты со статусом "вдовец / вдова" имеют самый низкий процент по невозврату кредитов.
Нет явной зависимости между уровнем дохода и возвратом кредита в срок.
Прослеживается зависимость между целью кредита и возвратом кредита в срок:
самые ответственные заемщики - это клиенты из категорий "операции с недвижимостью" и "проведение свадьбы".
хуже обстоят дела с категориями "операции с автомобилем" и "получение образования". Они чаще не возвращают кредиты.